In [1]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import string
from nltk.corpus import stopwords
plt.ion()

In [17]:
class SentimentAnalysis:
    def __init__(self):
        
        self.map_word = {}
        self.map_index = {}
        self.x_y_pairs = {}
        self.pairs_map = {}
    def CBOW(self):
        pass
    
    def word2vec(self):
        pass
    
    def NGRAM(self):
        pass

In [13]:
%time
def clean_data(reviews):
    reviews.text = reviews.text.apply(lambda x: x.lower())
    
    def remove_punctuation(s):
    #s = s.split()
    
        s = [ch for ch in s if ch not in string.punctuation]

        s = "".join(s)

        return s
    
    
    def remove_stopwords(s):
        stop_words = set(stopwords.words("english"))

        s = s.split(" ")

        s = [word for word in s if word not in stop_words]

        s = " ".join(s)

        return s
    
    
    reviews.text = reviews.text.apply(lambda x:remove_punctuation(x))
    reviews.text = reviews.text.apply(lambda x:remove_stopwords(x))
    
    return reviews

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [4]:
%%time
reviews = pd.read_json("/home/bh_parijat/data/yelp/review.json",lines=True)

CPU times: user 1min 29s, sys: 27.8 s, total: 1min 57s
Wall time: 1min 54s


In [5]:
%%time
reviews.head()

CPU times: user 447 µs, sys: 0 ns, total: 447 µs
Wall time: 374 µs


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [6]:
%%time
reviews = clean_data(reviews)

CPU times: user 20min 29s, sys: 54.1 s, total: 21min 23s
Wall time: 21min 24s


In [7]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,total bill horrible service 8gs crooks actuall...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,adore travis hard rocks new kelly cardenas sal...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,say office really together organized friendly ...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,went lunch steak sandwich delicious caesar sal...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,today second three sessions paid although firs...,2018-01-30 23:07:38


In [8]:
reviews.label = reviews.stars.apply(lambda x: 1 if x>=3 else 0)

In [14]:
def tokenize_vocab(map_word,map_index,x):
    
    idx = len(map_word.keys())
    ans = []
    for word in x.split():
        if word not in map_word:
            idx+=1
            map_word[word] = idx
            map_index[idx] = word
        ans.append(idx)
            
    return ans


def build_context_center(x,pairs_map,window = 2):
    
    if window not in pairs_map:
        pairs_map[window] = []
        
    x = x.split()
    for i in range(len(x)):
        
        for j in range(1,window+1):
            
            if (i-j)>0:
                pairs_map[window].append([x[i],x[i-j]])
                
            if (i+j)<n:
                pairs_map[window].append([x[i],x[i+j]])
                
                

In [ ]:
%%time
sla = SentimentAnalysis()
reviews['tokenized'] = reviews.text.apply(lambda x: tokenize_vocab(sla.map_word,sla.map_index,x))

In [ ]:
_ = reviews.tokenized.apply(lambda x:build_context_center(x,sla.pairs_map))